In [1]:
import numpy as np
from queue import PriorityQueue as queue
from collections import defaultdict
from faker import Faker
from tabulate import tabulate
from collections import deque, Counter

fake = Faker()

## Draw Simulation Methods

In [2]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = {-1: 'Bye'}, {-1: 1600}
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append([p_id, p_name, p_rank])
        id_to_name[p_id] = p_name
        id_to_rank[p_id] = p_rank
    return players, id_to_name, id_to_rank

In [3]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [4]:
def run_round(pairings, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    BYE = -1
    for pairing in pairings:
        a, b = pairing[0][1], pairing[1][1]
        if a == BYE:
            result = [a,b,b]
        elif b == BYE:
            result = [a,b,a]
        else:
            result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result)
        
    return results

## Draw Calculation Methods

In [5]:
def create_draw(players, results):
    """
    Create draw for the round
    Agruments:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
        results: List of pairings and results for all rounds to date
            (m x 2) list containing [player_a_id, player_b_id, winner_id]
    Returns:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        splits: List of index points where each points bracket finishes
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairings: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(players)
    add_bye = n_players % 2 == 1
    
    standings       = get_standings(results, add_bye=add_bye)
    scores, splits  = get_scores(standings, results)
    alloc, pairings = get_allocations(standings, scores)
    
    return standings, splits, scores, alloc, pairings

In [6]:
def get_standings(results, add_bye=False):
    """
    Calculate player standings at start of round
    Arguments:
        results: List of pairings and results for all rounds to date
            (m x 2) list containing [player_a_id, player_b_id, winner_id]
    Returns:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
    """
    BYE = -1
    standings = defaultdict(int)
    if len(results) > 0:
        for result in results:
            standings[result[0]] += (result[0] == result[2])
            standings[result[1]] += (result[1] == result[2])
    else:
        for player in players:
            standings[player[0]] = 0

    if add_bye:
        mode = np.argmax(np.bincount(list(standings.values()))) if len(results) > 0 else 0
        standings[BYE] = mode
    elif BYE in standings:
        del(standings[BYE])

    standings = [[k, v] for k, v in standings.items()]
    standings = np.array(sorted(standings, key=lambda x: x[1], reverse=True))
    
    return standings

In [7]:
def get_scores(standings, pairs):
    """
    Assign scores to each possible pairing
    A higher score means that the pairing is more strongly preferred
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        pairs: List of pairings for all rounds to date
            (m x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        splits: List of index points where each points bracket finishes
    """
    n_players = len(standings)
    ix = {player_id: pos for pos, player_id in enumerate(standings[:,0])}
    splits = get_splits(standings)
    
    # Calculate scores for pairings
    scores = np.full((n_players, n_players), -100)
    
    split_1, split_2 = 0, 0
    for split in splits + [n_players]:
        # Assign points for players in same bracket
        scores[split_1:split, split_1:split] = 16
        # Assign points for pull-up bracket
        if split_1 %2 == 1:
            scores[split_2:split_1, split_1:split] = 1
            scores[split_1:split, split_2:split_1] = 1
        split_1, split_2 = split, split_1
    
    # Reset self vs self to -inf
    np.fill_diagonal(scores, -100)

    # Subtract points for prior matchup
    BYE = -1
    for pair in pairs:
        a, b = ix[pair[0]], ix[pair[1]]
        penalty = 32 if BYE in set(pair) else 8
        scores[a,b] -= penalty
        scores[b,a] -= penalty
        
    return scores, splits

In [8]:
def swap_pairs(pair_1, pair_2, alloc):
    """
    Helper function to swap players between two pairings
    Original pairing of (a,b) and (c,d) becomes (a,c) and (b,d)
    Arguments:
        pair_1: Tuple containing first pair to swap (a,b)
        pair_2: Tuple containing first pair to swap (c,d)
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
    Returns:
        alloc: The matrix form pairing allocations with [(a,b), (c,d)] -> [(a,c), (b,d)]
            (n x n) matrix of integers        
    """
    a, b = pair_1
    c, d = pair_2

    alloc[a, b] = 0
    alloc[b, a] = 0
    alloc[c, d] = 0
    alloc[d, c] = 0
    alloc[a, c] = 1
    alloc[c, a] = 1
    alloc[b, d] = 1
    alloc[d, b] = 1
    
    return alloc

In [9]:
def get_splits(standings):
    """
    Helper method to find the points at which the brackets split
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
    """
    n_players = len(standings)
    ix = np.argsort(standings[:,1])[::-1]
    standings = standings[ix]
    splits = [n for n in range(n_players) if standings[n,1] != standings[n-1,1]] + [n_players]

    return splits

In [10]:
def get_allocations(standings, scores):
    """
    Split table down into even length brackets and sub-allocate
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
    Returns:
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairs: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(standings)
    splits = get_splits(standings)

    # Allocate entire table if there is no split
    if len(splits) == 1:
        return _get_allocations(standings, scores)

    allocs = np.zeros((n_players, n_players))
    pairs = []

    # Split into sub-tables at even numbered index split points
    ixs = np.array(splits)
    ixs = ixs[ixs % 2 == 0]
    for ix1, ix2 in zip(ixs[:-1], ixs[1:]):
        alloc, pair = _get_allocations(standings[ix1:ix2], scores[ix1:ix2,ix1:ix2], start=ix1)

        allocs[ix1:ix2,ix1:ix2] = alloc
        pairs += pair

    return allocs, pairs

def _get_allocations(standings, scores, start=0):
    """
    Allocate player pairings
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
    Returns:
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairs: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(standings)
    ix = {pos+start: player_id for pos, player_id in enumerate(standings[:,0])}
    alloc = np.zeros((n_players, n_players), dtype=int)
    splits = get_splits(standings)

    # Construct initial draw
    for _ in range(int(n_players / 2)):
        mask = (1 - alloc.sum(axis=1))
        # Choose Player A
        p = scores.max() - scores.mean(axis=1)
        p = p * mask
        a = p.argmax()
        # Choose Player B
        p = np.exp(scores[a])
        p = np.maximum(p * mask, 0)
        b = np.random.choice(range(n_players),p=p/p.sum())
        
        alloc[a, b] = 1
        alloc[b, a] = 1

    # Run Metropolis-Hastings to iteratively improve draw
    prob = np.exp(scores)
    prob /= prob.sum(axis=1, keepdims=True)

    score_q = [(alloc*scores).sum()/len(prob)]
    best_alloc = alloc.copy()
    max_score = 16 - 15 * (np.array(splits) % 2).sum() * 2 / len(scores)

    while len(score_q) < 10 or np.mean(score_q[-5:]) > np.mean(score_q[-10:-5]):        
        if max(score_q) == max_score:
            break
        for a, p in enumerate(prob):
            b = alloc[a].argmax()
            c = np.random.choice(range(len(p)),p=p)
            d = alloc[c].argmax()
            p_change = prob[a,c]*prob[b,d]/(prob[a,b]*prob[c,d] + prob[a,c]*prob[b,d])
            if np.random.random() < p_change:
                alloc = swap_pairs((a,b), (c,d), alloc)

        score = (alloc*scores).sum()/len(prob)
        if score > max(score_q):
            best_alloc = alloc.copy()
        score_q.append(score)
    
    alloc = best_alloc
    del(best_alloc)
    
    # Create list form of pairings
    alloc_ = alloc * np.tri(*alloc.shape)

    pairs = []
    for n in range(n_players):
        if alloc_[n].max() == 1:
            a, b = n + start, alloc_[n].argmax() + start
            pairs.append([(a, ix[a]), (b, ix[b])])

    return alloc, pairs

## Draw Printing Methods

In [28]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([int(player[0]), id_to_name[player[0]], player[1], rank+1])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Rank']), '\n')
    

def print_pairings(pairs, id_to_name, scores):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        b, a = pair[0], pair[1]
        score = scores[b[0], a[0]]
        row = [table_num+1, id_to_name[a[1]], int(a[1]), int(a[0])+1, id_to_name[b[1]], int(b[1]), int(b[0])+1, score]
        table.append(row)
    headers = ['Table #', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank', 'GOF']
    print('\n', tabulate(table, headers=headers), '\n')

def print_byes(pairs):
    byes = [pair[[1,3]] for pair in np.reshape(pairs_h, (-1, 4)) if -1 in pair]
    byes = Counter(np.reshape(byes, -1))
    print('----- BYES -----')
    for k, v in byes.items():
        if k == -1:
            print('Byes', ': ', v)
        else:
            print(k, ': ', v)

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes
- Allocating pods
- Iterative sampling methods to improve draw quality
- etc

In [30]:
NUMBER_OF_PLAYERS = 11
NUMBER_OF_ROUNDS  = 6
PROBABILITY_OF_DRAW = 0.0001

In [31]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)

In [32]:
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []
number_of_players = NUMBER_OF_PLAYERS + NUMBER_OF_PLAYERS % 2

for rnd in range(NUMBER_OF_ROUNDS):
    standings, splits, scores, alloc, pairs = create_draw(players, results)
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / number_of_players)
    print('Maximum GOF:', 16 - 15 * (np.array(splits) % 2).sum() * 2 / number_of_players)
    print("Frequencies: " + ', '.join([str(int(k)) + "pts: " + str(int(v / 2)) 
                                 for k, v in Counter((scores * alloc).reshape(-1)).items() if k != 0]) )
    print_standings(standings, id_to_name, )
    print_pairings(pairs, id_to_name, scores)
    result = run_round(pairs, p_draw=PROBABILITY_OF_DRAW)
    results += result

print('Overall Pairing Score Frequency: ', ', '.join([str(int(k)) + "pts: " + str(int(v / 2))
   for k, v in Counter((np.array(scores_h)*np.array(alloc_h))[-NUMBER_OF_ROUNDS:].reshape(-1)).items() if k != 0]))
print_byes(pairs_h)

----- Round 1 -----
Goodness of Fit: 16.0
Maximum GOF: 16.0
Frequencies: 16pts: 6

      ID  Name                Wins    Rank
-------  ----------------  ------  ------
7077685  Casey Taylor           0       1
8113708  Robert Alexander       0       2
8821747  Jennifer Jones         0       3
5332388  David Smith            0       4
1033668  Mrs. Susan Huber       0       5
2114274  Tracy Morris           0       6
3371738  Julie Berry            0       7
9343117  Bradley Sullivan       0       8
2842174  Gary Meza              0       9
5538038  Jessica Morgan         0      10
1499534  Adrienne Jones         0      11
     -1  Bye                    0      12 


   Table #  Player 1 - Name         ID    Rank  Player 2 - Name         ID    Rank    GOF
---------  -----------------  -------  ------  -----------------  -------  ------  -----
        1  David Smith        5332388       4  Mrs. Susan Huber   1033668       5     16
        2  Robert Alexander   8113708       2  Tracy Morr